In [2]:
import torch
import random
import numpy as np

SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
# Import pretrained transformer
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [5]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

<s> </s> <pad> <unk>


In [6]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

0 2 1 3


In [7]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id
print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

0 2 1 3


In [8]:
maxInputLength = tokenizer.max_model_input_sizes['roberta-base']
print(maxInputLength)

512


In [9]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:maxInputLength-2]
    return tokens

In [10]:
from torchtext.legacy import data
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [11]:
from torchtext.legacy import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

downloading aclImdb_v1.tar.gz


D:\New_\.data\imdb\aclImdb_v1.tar.gz: 100%|███████████████████████████████████████| 84.1M/84.1M [00:19<00:00, 4.26MB/s]


In [12]:
LABEL.build_vocab(train_data)
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)


from transformers import RobertaTokenizer, RobertaModel
import torch.nn as nn
rob = RobertaModel.from_pretrained('roberta-base')

class RobertaSentiment(nn.Module):
    def __init__(self,
                 rob,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        self.rob = rob
        embedding_dim = rob.config.to_dict()['hidden_size']
        # Define rnn
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        with torch.no_grad():
        _, thehiddenlayer = self.rnn(self.rob(text)[0])
        if self.rnn.bidirectional:
            thehiddenlayer = self.dropout(torch.cat((thehiddenlayer[-2,:,:], thehiddenlayer[-1,:,:]), dim = 1))
        else:
            thehiddenlayer = self.dropout(thehiddenlayer[-1,:,:])
        
        output = self.out(thehiddenlayer)
        return output

100%|████████████████████████████████████████████████████████████████| 501200538/501200538 [01:11<00:00, 7057379.41B/s]


In [13]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = RobertaSentiment(rob,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)


def countPra(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {countPra(model):,} trainable parameters')

for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False
        
def reCountPra(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {reCountPra(model):,} trainable parameters')



The model has 127,404,801 trainable parameters
The model has 127,404,801 trainable parameters


In [14]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)


def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0 
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [15]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [16]:


import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs



In [17]:


N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'sentiment_model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 309m 53s
	Train Loss: 0.386 | Train Acc: 80.99%
	 Val. Loss: 0.224 |  Val. Acc: 91.75%
Epoch: 02 | Epoch Time: 295m 11s
	Train Loss: 0.203 | Train Acc: 92.13%
	 Val. Loss: 0.176 |  Val. Acc: 93.39%
Epoch: 03 | Epoch Time: 289m 45s
	Train Loss: 0.180 | Train Acc: 93.19%
	 Val. Loss: 0.188 |  Val. Acc: 93.03%
Epoch: 04 | Epoch Time: 286m 16s
	Train Loss: 0.180 | Train Acc: 93.08%
	 Val. Loss: 0.177 |  Val. Acc: 93.47%
Epoch: 05 | Epoch Time: 288m 49s
	Train Loss: 0.155 | Train Acc: 94.36%
	 Val. Loss: 0.178 |  Val. Acc: 93.65%


In [18]:
model.load_state_dict(torch.load('sentiment_model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.162 | Test Acc: 94.00%


In [19]:
def polarityDetection(content, inModel, inTokenizer):
    model.eval()
    tokens = tokenizer.tokenize(content)
    tokens = tokens[:maxInputLength-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    
    if prediction.item() > 0.5:
        return "Positive"
    elif prediction.item() < 0.5:
        return "Negative"
    else:
        return "Neutral"

In [20]:
# Clean up content
import pandas as pd
df = pd.read_json('clean.json')

import re

def cleanContentText(content):
    content = re.sub('[^A-Za-z]+', ' ', content)
    return content

df['Temp Content'] = df['content'].apply(cleanContentText)
import nltk
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet

labelDic = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N': wordnet.NOUN, 'R': wordnet.ADV}

def getSentiLabel(content):
    senTags = pos_tag(word_tokenize(content))
    labels = []
    for word, label in senTags:
        if word.lower() not in set(stopwords.words('english')):
            labels.append(tuple([word, labelDic.get(label[0])]))
    return labels

df['Word Labels'] = df['Temp Content'].apply(getSentiLabel)
from nltk.stem import WordNetLemmatizer
wordnetLem = WordNetLemmatizer()

def lemmatize(inLabels):
    result = ""
    for word, label in inLabels:
        if not label:
            result = result + " " + word
        else:
            temp = wordnetLem.lemmatize(word, pos = label)
            result = result + " " + temp
    return result

df['Cleaned Content'] = df['Word Labels'].apply(lemmatize)
df.drop(columns=['Word Labels', 'Temp Content'], inplace = True)
df.head()

,category,location,title,author,create_time,content,image,Cleaned content
0,singapore transport,singapore,lower octane petrol prices dip below 3 a litre,christopher tan,2022-04-04 12:19:38,singapore pump prices have fallen by five cent...,https://static1.straitstimes.com.sg/s3fs-publi...,singapore pump prices have fallen by five cent...
1,singapore,singapore,at least 1 month waiting time for new s'pore p...,isabelle liew,2022-04-04 10:21:45,singapore singaporeans who apply for a new pas...,https://static1.straitstimes.com.sg/s3fs-publi...,singapore singaporeans who apply for a new pas...
2,singapore housing,singapore,work on 9 hdb commercial complexes hit by dela...,michelle ng,2022-04-04 05:00:00,singapore work at eight housing board commerci...,https://static1.straitstimes.com.sg/s3fs-publi...,singapore work at eight housing board commerci...
3,singapore,singapore,online sales beef up family's butcher business...,kolette lim,2022-04-04 05:00:00,singapore she saw her parents struggling as bu...,https://static1.straitstimes.com.sg/s3fs-publi...,singapore she saw her parents struggling as bu...
4,singapore,singapore,undergrads adapt alcohol delivery offerings to...,kolette lim,2022-04-04 05:00:00,singapore when mr chai wan lin and mr tneoh yu...,https://static1.straitstimes.com.sg/s3fs-publi...,singapore when mr chai wan lin and mr tneoh yu...


In [21]:
# Predict sentiment result
df['Sentiment Label'] = df['Cleaned content'].apply(polarityDetection, inModel = model, inTokenizer = tokenizer)
df.head()

,category,location,title,author,create_time,content,image,Cleaned content,Sentiment Label
0,singapore transport,singapore,lower octane petrol prices dip below 3 a litre,christopher tan,2022-04-04 12:19:38,singapore pump prices have fallen by five cent...,https://static1.straitstimes.com.sg/s3fs-publi...,singapore pump prices have fallen by five cent...,Positive
1,singapore,singapore,at least 1 month waiting time for new s'pore p...,isabelle liew,2022-04-04 10:21:45,singapore singaporeans who apply for a new pas...,https://static1.straitstimes.com.sg/s3fs-publi...,singapore singaporeans who apply for a new pas...,Positive
2,singapore housing,singapore,work on 9 hdb commercial complexes hit by dela...,michelle ng,2022-04-04 05:00:00,singapore work at eight housing board commerci...,https://static1.straitstimes.com.sg/s3fs-publi...,singapore work at eight housing board commerci...,Positive
3,singapore,singapore,online sales beef up family's butcher business...,kolette lim,2022-04-04 05:00:00,singapore she saw her parents struggling as bu...,https://static1.straitstimes.com.sg/s3fs-publi...,singapore she saw her parents struggling as bu...,Positive
4,singapore,singapore,undergrads adapt alcohol delivery offerings to...,kolette lim,2022-04-04 05:00:00,singapore when mr chai wan lin and mr tneoh yu...,https://static1.straitstimes.com.sg/s3fs-publi...,singapore when mr chai wan lin and mr tneoh yu...,Positive


In [22]:
sentiResultStat = df['Sentiment Label'].value_counts()
sentiResultStat

Positive    13008
Negative      277
Name: Sentiment Label, dtype: int64